# ResultAnalysis
Notebook to check Grid Search results

In [1]:
import os
from working_dir import set_wd
set_wd()
os.getcwd()

'/Users/tales.pimentel/ds/kaggle/football-match-prediction'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from src.dao import dao_ml
from src.utils import dflib, stats, pretties, plot, plot_domain, palette

In [3]:
pretties.max_data_frame_columns()

# Loading Results

In [4]:
all_results = dao_ml.load_all_modeling()
print(len(all_results))

pd.DataFrame(all_results)[["id_modeling", "datetime", "clf_name", "pipeline_train_stages"]].sort_values("datetime", ascending=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/modeling/'

In [ ]:
load_ids_modeling = ["d93331f6-3b0c-4753-a522-8e941c2e8ac0", "7ce98dc7-7233-4833-89e4-85eb3eda53b9"]

rdf = pd.DataFrame()

for id_modeling in load_ids_modeling:
    results_temp = dao_ml.load_modeling(id_modeling)
    
    rdf_id = pd.DataFrame(results_temp["overfitting_analysis_df"])
    rdf_id["clf_name"] = results_temp["clf_name"]
    
    rdf = rdf.append(rdf_id)

In [ ]:
rdf = rdf.sort_values("log_loss_cv")
rdf.head(5)

In [ ]:
def overfitting_analysis_num(metrics_df, param_name, metric_colname):
    metric_colname_train = metric_colname + "_train"
    metric_colname_valid = metric_colname + "_cv"
    
    of_data = metrics_df.groupby(param_name)[[metric_colname_train, metric_colname_valid]].mean().reset_index()

    display(of_data)
    
    title = f"{metric_colname} mean values for {param_name}"
    
    plt.figure(figsize=(8,5))
    plt.scatter(of_data[param_name], of_data[metric_colname_train], label="train", s=130, alpha=0.6)
    plt.scatter(of_data[param_name], of_data[metric_colname_valid], label="cv", s=130, alpha=0.6)
    plt.title(label=title)
    plt.xlabel(param_name)
    plt.ylabel(metric_colname)
    plt.legend()
    plt.show()

In [ ]:
pretties.md("RandomForestClassificationModel", size="#", color="blue")
rfc_df = rdf[rdf["clf_name"] == "RandomForestClassificationModel"]

for param_name in ["subsamplingRate", "maxDepth", "numTrees"]:
    pretties.md(param_name, size="####")
    overfitting_analysis_num(metrics_df=rfc_df, 
                             param_name=param_name, 
                             metric_colname="log_loss")

In [ ]:
pretties.md("XGBClassifier", size="#", color="blue")
xgbc_df = rdf[rdf["clf_name"] == "XGBClassifier"]

for param_name in ["colsample_bytree", "max_depth", "n_estimators", "subsample"]:
    pretties.md(param_name, size="####")
    overfitting_analysis_num(metrics_df=xgbc_df, 
                             param_name=param_name, 
                             metric_colname="log_loss")